# 0 을 완료하기 위해서는 1을 끝내야 한다는 것을 [0,1] 쌍으로 표현하는 n 개의 코스가 있다. 
# 코스 개수 n 과 이 쌍들을 입력으로 받았을 때 모든 코스가 완료 가능한지 판별하라.

- 입력
```python
2, [[1,0]]
```
- 출력
```python
true
```

- 설명
    - 2개의 코스가 있으며, 1을 완료하기 위해 0을 끝내면 된다. 따라서 가능하다.

- 입력

```python
2, [[1,0],[0,1]]
```

```python
false
```
- 설명
    - 2개의 코스가 있으며, 1을 완료하기 위해 0을 끝내야 하고 0을 끝내기 위해서는 1을 끝내야 한다. 따라서 불가능하다.


In [4]:
    # for x,y in prerequisites: # 페어들의 목록인 prerequisites 를 풀어서 그래프로 표현  [1,0] = (x,y)
    #     # 페어의 첫번쨰 값이 x, 두번쨰 값이 y 로 놓고 x: key 에 복수 개의 y가 들어갈 수 있도록 함
    #     graph[x].append(y)
    # print(graph)
    # # traced = set()
canFinish(2, [[1,0],[2,1]])

defaultdict(<class 'list'>, {1: [0], 2: [1]})


1. 순환구조를 판별하기 위해 앞서 이미 방문했던 노드들을 traced 변수에 저장한다.
2. 이미 방문했던 곳을 중복 방문하게 된다면 순환구조로 간주할 수 있고, 이 경우 False 를 리턴하고 종료할 수 있다.
3. traced 는 중복값을 가지지 않으므로 set() 으로 정한다.
4. 여기서 DFS 함수인 dfs() 에서 현재 노드가 이미 방문했던 노드 집합인 traced 에 존재한다면 false 를 리턴한다.
5. false 는 계속 상위로 리턴되어 최종 결과도 false 를 리턴하게 될 것이다.
6. 탐색은 재귀로 진행하되, 해당 노드를 이용한 모든 탐색이 끝나게 된다면 traced.remove(i) 로 방문했던 내역을 반드시 삭제해야 한다.
7. 그렇지 않다면 형제 노드가 방문한 노드까지 남게 되어, 자식 노드 입장에서는 순환이 아닌데 순환이라고 잘못 판단할 수 있기 때문이다.

In [1]:
# DFS 로 순환 구조 판별하기
from typing import List
import collections

def canFinish(n:int, prerequisites:List[List[int]])->bool:
    graph = collections.defaultdict(list)
    # 그래프 구성
    for x,y in prerequisites: # 페어들의 목록인 prerequisites 를 풀어서 그래프로 표현  [1,0] = (x,y)
        # 페어의 첫번쨰 값이 x, 두번쨰 값이 y 로 놓고 x: key 에 복수 개의 y가 들어갈 수 있도록 함
        graph[x].append(y)
    traced = set()

    def dfs(i):
        # 순환구조이면 False
        if i in traced:
            return False
        traced.add(i)
        for y in graph[i]:
            if not dfs(y):
                return False
        # 탐색 종료 후 순환 노드 삭제
        traced.remove(i)

        return True
    for x in list(graph): # 끝까지 False 가 검출되지 않았다면 True 를 return
        if not dfs(x):
            return False
    return True

canFinish(2, [[1,0],[0,1]])

## 경로

1. 그래프를 설정  "graph = defaultdict(<class 'list'>, {1: [0], 0: [1]})"
    - for x in list(graph) 문이 실행된다. x 는 1부터 시작한다.
2. dfs(1) 을 실행 -> traced 에 경로 "1"이 추가된다.
3. for y in graph[i] 에서 i=1 이므로 graph[1] = 0 이다. if not dfs(0) 에서 dfs(0)이 재귀호출된다.
4. dfs(0) 을 실행 -> traced 에 경로 "0" 이 추가된다. traced 에 경로가 중복되지 않았으므로 코드는 계속 실행된다.
5. for y in graph[i] 에서 i=0 이므로 graph[0] = 1 이다. if not dfs(1) 에서 dfs(1) 이 재귀호출된다.
6. dfs(1) 을 실행 -> traced 에 경로 1이 존재하므로 False 를 return 한다.
7. i = 0 일 때 작동했던 if not dfs(1): 이 실행된다. if not False 이므로 if True 가 되어 이하의 문장이 실행된다.
8. False 를 return 한다. 
9. 마지막으로 i = 1 까지 돌아와 if not dfs(0) 이 실행되는데, dfs(0)이 false 이므로 다시 false 를 리턴한다.
10. x = 1 에서 false 가 이미 리턴되었으므로 반복문을 종료한다.  

In [ ]:
# 가지치기로 최적화하기
# 앞선 DFS 풀이는 순환이 발견될 떄까지 모든 자식노드를 탐색하는 구조로 되어있다. 
# 만약 순환이 아니더라도 서로 복잡하게 호출하는 구조로 그래프가 구성되어 있다면, 불필요하게 동일한 그래프를 여러 번 탐색하게 될 수도 있다.
# 따라서 한번 방문했던 그래프는 두 번 이상 방문하지 않도록 무조건 True 로 리턴하는 구조로 개선한다면, 탐색시간을 획기적으로 줄일 수 있다.

# 가지치기 = 불필요한 노드 스킵

from typing import List
import collections

def canFinish(n:int, prerequisites:List[List[int]])->bool:
    graph = collections.defaultdict(list)
    # 그래프 구성
    for x,y in prerequisites: 
        graph[x].append(y)

    traced = set()
    visited = set()

    def dfs(i):
        # 순환구조이면 False
        if i in traced:
            return False
        # 이미 방문했던 노드이면 True
        if i in visited:
            return True
        

        traced.add(i)
        for y in graph[i]:
            if not dfs(y):
                return False
        # 탐색 종료 후 순환 노드 삭제
        traced.remove(i)
        # 탐색 종료 후 방문 노드 추가
        visited.add(i)
        # visited 면 true 인게 어떻게 가지치기로 작동하는가? 
        ## 그냥 visited.add(i) 가 전부 false 가 아닐 때 작동하니까, 이미 visited 가 true 임을 증명하는 경로와 같기 떄문이다.

        return True
    
    # 순환 구조 판별
    for x in list(graph): # 끝까지 False 가 검출되지 않았다면 True 를 return
        if not dfs(x):
            return False
    return True

canFinish(2, [[1,0],[0,1]])

- 여기서 visited 는 모든 탐색이 끝난 뒤 노드를 추가하는 형태로 구현한다. 
- 만약 탐색 도중 순환 구조가 발견된다면 False 를 return 하면서 visited 추가는 하지 않음은 물론, 모든 함수를 빠져나가며 종료하게 될 것이다. 
- 이렇게 한 번 방문한 노드는 더 이상 탐색하지 않는 형태, 즉 가지치기로 최적화된 풀이가 된다.